In [ ]:
!pip install -U tqdm music21

     |████████████████████████████████| 71kB 2.5MB/s 
     |████████████████████████████████| 18.5MB 161kB/s 
  Created wheel for music21: filename=music21-5.7.2-cp36-none-any.whl size=22024602 sha256=aa713d8e0096caf718696c662ebb87b9cfc270f2f1f7ddbe88579ad55414047c
  Stored in directory: /root/.cache/pip/wheels/6e/d0/05/1ef3daa9ae295073d807e468fcd820641965086424f1c633e3
Successfully built music21
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Found existing installation: music21 5.5.0
    Uninstalling music21-5.5.0:
      Successfully uninstalled music21-5.5.0


In [ ]:
!pip install -U google-api-python-client oauth2client PyDrive

     |████████████████████████████████| 61kB 2.2MB/s 
Requirement already up-to-date: oauth2client in /usr/local/lib/python3.6/dist-packages (4.1.3)
Requirement already up-to-date: PyDrive in /usr/local/lib/python3.6/dist-packages (1.3.1)
     |████████████████████████████████| 92kB 5.5MB/s 
     |████████████████████████████████| 1.3MB 13.2MB/s 
ERROR: google-api-core 1.21.0 has requirement google-auth<2.0dev,>=1.18.0, but you'll have google-auth 1.17.2 which is incompatible.
  Found existing installation: protobuf 3.10.0
    Uninstalling protobuf-3.10.0:
      Successfully uninstalled protobuf-3.10.0
  Found existing installation: google-api-core 1.16.0
    Uninstalling google-api-core-1.16.0:
      Successfully uninstalled google-api-core-1.16.0
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12


In [ ]:
import os
from glob import glob
from tqdm import tqdm
from functools import reduce
from music21 import converter, instrument, note, chord, stream
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import keras
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from keras.utils import normalize
from keras.models import Sequential, Model, model_from_json
from keras.layers import Input, Dense, Dropout, Activation, LSTM, Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D, RepeatVector, UpSampling1D
from keras import backend as K
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


###  Bajada de archivos de Gdrive


In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


def download_from_drive(local_fn, id):
  print('Downloading: %s, id: %s' % (local_fn, id))
  downloaded = drive.CreateFile({'id': id})
  downloaded.GetContentFile(local_fn)

fns = [
    ('LSTMSinRegularizanTrTe.h5', '1dglMb7yK-tMocu5ufoyz3ofT5IeLzd5L'), 
    ('LSTMSinRegularizanTrTe.json', '1cI3-BiytVBsBAVISKwYdgCe_Zd6ArY7D'), 
    ('LSTMAutoencoder.h5', '1M1YIRGvhtDPWGKzwjldxTD_yrCWsaeWd'), 
    ('LSTMAutoencoder.json', '1SIZWIIim6wpJ4GW0Kqq2aVFajRVr0zFi'), 
    ('bach.tar.gz', '179tgDp-U3oPf0DuCIDuA59-L9nfhqBJN'), 
]

for local_fn, id in fns:
  download_from_drive(local_fn, id)

Downloading: LSTMSinRegularizanTrTe.h5, id: 1dglMb7yK-tMocu5ufoyz3ofT5IeLzd5L
Downloading: LSTMSinRegularizanTrTe.json, id: 1cI3-BiytVBsBAVISKwYdgCe_Zd6ArY7D
Downloading: LSTMAutoencoder.h5, id: 1M1YIRGvhtDPWGKzwjldxTD_yrCWsaeWd
Downloading: LSTMAutoencoder.json, id: 1SIZWIIim6wpJ4GW0Kqq2aVFajRVr0zFi
Downloading: bach.tar.gz, id: 179tgDp-U3oPf0DuCIDuA59-L9nfhqBJN


In [ ]:
!tar -xzf bach.tar.gz

## Práctica Guiada - Autoencoders con midi files

### Introducción: Archivos Midi

En esta práctica vamos a utilizar autoencoders para codificar y decodificar archivos de audio de tipo midi, Musical Instrument Digital Interface.
MIDI es un protocolo de representación de música, que cuenta con una importante difusión.

A diferencia de los archivos de audio que venimos trabajando en el resto de este módulo, como los .wav ó .mp3, los archivos midi <strong> no contienen una señal de audio sino las instrucciones necesarias para generarlo </strong>. 

En este sentido, el contenido de un archivo midi puede entenderse como una secuencia de instrucciones que opera sobre distintos canales: uno por cada instrumento que deberá generar la música. En total, los archivos midi pueden mandar instrucciones por un máximo de 16 canales, permitiendo entonces, 16 instrumentos simultáneamente.

### Estructura de un archivo Midi

Los archivos Midi almacenan la secuencia de instrucciones en un formate de nota/time_delta. Es decir, se guarda, para cada instrumento, las notas a tocar y el tiempo hasta la siguiente instrucción.
<br>
<img src='https://image.ibb.co/c1Z8E7/Selecci_n_126.png'></img>

La librería de Python music21 permite parsear archivos de tipo midi y trabajar con objetos que representan las instrucciones de forma más clara, como por ejemplo, nota, acorde e instrumento. Cuando un conjunto de notas se debe reproducir simultáneamente, music21 lo parsea como un acorde. Cada nota y acorde tiene asociado un offset que indica en qué momento de la reproducción del audio se lo debe tocar. 

## Preprocesamiento de datos


Escribimos una función que reciba lea un archivo midi y almacene en una lista la secuencia de notas y acordes. Por simplicidad, en esta práctica vamos a trabajar exclusivamente con música de piano, pero se podría trabajar con más instrumentos utilizando varios canales. 



In [ ]:
def get_notes(audio_file):
    midi = converter.parse(audio_file)
    instrument_partition = instrument.partitionByInstrument(midi)
    audio_features = instrument_partition.parts[0].recurse()
    
    def get_note_chord(audio_feature):
        if isinstance(audio_feature, note.Note):
            return str(audio_feature.pitch)
        elif isinstance(audio_feature, chord.Chord):
            return '.'.join(map(lambda x: str(x), audio_feature.normalOrder))
    
    return list(filter(lambda x: x!= None, map(get_note_chord, audio_features)))

Leamos ahora los archivos que componen nuestro input:

In [ ]:
# Trabajamos con los primeros 400 archivos para aligerar el procesamiento
nmidis = 400
audio_files = sorted(glob("./bach/*.mid"), key=os.path.getsize)[0:nmidis]

In [ ]:
len(audio_files)

400

Con estos audios y la función anterior, creamos una lista de listas que contenga la secuencia de instrucciones para cada audio:

In [ ]:
midis_notes_chords_by_file = []

for file in audio_files:
    try:
        notes_file = get_notes(file)
        midis_notes_chords_by_file.append(notes_file)
    except Exception as err:
        print(file, err)

./bach/458b2d957a32475619d157902a430866.mid 'NoneType' object has no attribute 'parts'
./bach/b92f0db2ea8e70515dddd1d73fbcfeb1.mid 'NoneType' object has no attribute 'parts'
./bach/60fc529f381b1ce1aaccc313215f9fb4.mid 'NoneType' object has no attribute 'parts'
./bach/19cf29a73f93f11c14972f647b3bdc4e.mid 'NoneType' object has no attribute 'parts'
./bach/24e3ce35ec2f4fcf2cd1adc296799810.mid 'NoneType' object has no attribute 'parts'
./bach/bc66a983c37d0dce783d90fa183ee739.mid 'NoneType' object has no attribute 'parts'
./bach/0f2a3840813cf4fe6e22dcf73666e12a.mid 'NoneType' object has no attribute 'parts'
./bach/327852b1e95de5032bb1db6f7738045f.mid 'NoneType' object has no attribute 'parts'
./bach/11c8c6bd558ce7512d454e9a395f1fb7.mid 'NoneType' object has no attribute 'parts'


In [ ]:
len(midis_notes_chords_by_file)

391

Para simplificar el trabajo de los autoencoders (y de paso simplificar el conjunto de entrenamiento), vamos a dividir cada audio en chunks de un máximo de 10 instrucciones cada uno:

In [ ]:
timesteps = 10

In [ ]:
def get_chunks(file_notes):
    new_chunks = []
    num_chunks = len(file_notes) // timesteps
    for n in range(0,num_chunks):
        new_chunks.append(file_notes[n*timesteps:n*timesteps + timesteps])
    return new_chunks

In [ ]:
all_chunks = []

In [ ]:
for file in midis_notes_chords_by_file:
    all_chunks = all_chunks + get_chunks(file)

In [ ]:
len(all_chunks)

18335

In [ ]:
all_chunks[0]

['C5', 'B4', 'C5', 'G4', 'G#4', 'F4', 'B4', 'C5', 'D5', 'G4']

Una vez que definimos que nuestros audios van a ser unidades de 10 instrucciones, tendremos que construir los arrays necesarios para alimentar a un autoencoder.

La cantidad 10 representa los pasos de nuestra secuencia, así que vamos a conservarla en el eje Y.

Nuestros samples están dados por todos los chunks almacenados en la lista all_chunks.
La listas de all_chunks contienen descripciones de los datos. Para alimentar a la red neuronal vamos a transformar estas representaciones a números.
Para esto vamos a utilizar la clase LabelEncoder de sklearn.

In [ ]:
# Pasamos a numpy array
array_input = np.array(all_chunks)

# Instanciamos el LabelEncoder
le = LabelEncoder()

# El LabelEncoder "aprende" todas las clases que existen. 
# El método fit de LE exige un input unidimensional, por eso hacemos reshape.
le.fit(array_input.reshape(len(all_chunks)*timesteps))

# Usamos el diccionario aprendido por LabelEncoder para transformar un array de dos dimensiones
array_input = le.transform(array_input.reshape(len(all_chunks)*timesteps))

# Normalizamos el input para una NN
aray_input = array_input/len(le.classes_)

# Agregamos la dimensión de canal (un único instrumento)
array_input = array_input.reshape(len(all_chunks),timesteps,1)

In [ ]:
# Verificamos la shape resultante
array_input.shape

(18335, 10, 1)

In [ ]:
# Vemos algunos ejemplos
array_input[:5]

array([[[216],
        [205],
        [216],
        [253],
        [247],
        [241],
        [205],
        [216],
        [222],
        [253]],

       [[216],
        [205],
        [216],
        [222],
        [241],
        [253],
        [247],
        [253],
        [241],
        [226]],

       [[253],
        [193],
        [205],
        [222],
        [216],
        [216],
        [232],
        [222],
        [222],
        [254]],

       [[237],
        [254],
        [222],
        [205],
        [253],
        [193],
        [205],
        [216],
        [222],
        [232]],

       [[222],
        [216],
        [205],
        [193],
        [205],
        [253],
        [236],
        [193],
        [221],
        [236]]])

## Ajustamos el autoencoder

En primer lugar vamos a ajustar un autoencoder básico. Vamos a trabajar con un encoder y un decoder básicos de 3 capas convolucionales. Utilizamos convoluciones 1D para captar las propiedades de las secuencias de instrucciones. 

In [ ]:
timesteps = 10

In [ ]:
from keras.layers import Input, Dense, MaxPooling1D, UpSampling1D
from keras.models import Model
from keras import backend as K

input_aud = Input(shape=(timesteps, 1))

x = Conv1D(32, 2, activation='relu', padding='same')(input_aud)
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(16, 2, activation='relu', padding='same')(x)
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(16, 2, activation='relu', padding='same')(x)
encoded = MaxPooling1D(2, padding='same')(x)

# En este punto, la representacion es (4, 4, 8), o sea, 128-dimensional

x = Conv1D(16, 3, activation='relu', padding='same')(encoded)
x = UpSampling1D(2)(x)
x = Conv1D(16, 3, activation='relu', padding='same')(x)
x = UpSampling1D(2)(x)
x = Conv1D(32, 4, activation='relu')(x)
x = UpSampling1D(2)(x)

decoded = Conv1D(1, 2, activation='sigmoid', padding='same')(x)

autoencoder = Model(input_aud, decoded, name='Autoencoder')
autoencoder.compile(optimizer='Adam', loss='binary_crossentropy')

In [ ]:
autoencoder.summary()

Model: "Autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 1)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 10, 32)            96        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 5, 32)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 5, 16)             1040      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 3, 16)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 3, 16)             528       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 2, 16)             

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(array_input, array_input, random_state=42, test_size = 0.2)

In [ ]:
x_train.shape

(14668, 10, 1)

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=300,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),)

Train on 14668 samples, validate on 3667 samples
Epoch 1/300
14668/14668 [==============================] - 2s 168us/step - loss: -107326960.7371 - val_loss: -1145259291.7153
Epoch 2/300
14668/14668 [==============================] - 1s 43us/step - loss: -136545470041.2195 - val_loss: -723218975151.7164
Epoch 3/300
14668/14668 [==============================] - 1s 47us/step - loss: -7897337254550.3740 - val_loss: -26813978024949.6680
Epoch 4/300
14668/14668 [==============================] - 1s 48us/step - loss: -120204045951496.5312 - val_loss: -305591180094481.5625
Epoch 5/300
14668/14668 [==============================] - 1s 56us/step - loss: -879252413804904.7500 - val_loss: -1842638216126658.2500
Epoch 6/300
14668/14668 [==============================] - 1s 50us/step - loss: -4113047722355568.5000 - val_loss: -7542744100779886.0000
Epoch 7/300
14668/14668 [==============================] - 1s 43us/step - loss: -14326313000616164.0000 - val_loss: -23861732268123016.0000
Epoch 8/300

## Analizando los resultados

¿Qué podemos decir acerca de la cantidad de parámetros del modelo y el número de datos de entrada? ¿Correspondería agregar un término de regularización?

Llevemos ahora los audios de entrenamiento y su correspondiente reconstrucción del autoencoder a un archivo midi para poder escucharlos. 

In [ ]:
reconstruccion = autoencoder.predict(x_train)

In [ ]:
# Hacemos una función que reconstruye la nomenclatura de las notas para los archivos midi
def get_notas(X):
    desnorm = (X*len(le.classes_)).astype(int)
    return list(le.classes_[desnorm-1].ravel()) # BORRAR Le agregué el -1 para que no pincharan los índices

Grabemos el primero de los audios

In [ ]:
prediction_output = get_notas(reconstruccion[0])

In [ ]:
offset = 0
output_notes = []
#  Creamos las notas en base a los patrones originales
for pattern in prediction_output:
    
    # Si el patrón es un acorde
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    
    # Si el patrón es una nota
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    
    # En cualquier caso, incrementamos el offset para manejar el tiempo de ejecución.
    offset += 0.7

In [ ]:
!mkdir output_midi
# Grabamos el archivo en una carpeta output_midi
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='output_midi/reconstruccion.mid')

mkdir: cannot create directory ‘output_midi’: File exists


'output_midi/reconstruccion.mid'

### Autoencoder Sequence to Sequence con LSTM

Ahora vamos a probar un autoencoder diseñado para mapear de secuencia a secuencia. 
Siguiento la arquitectura propuesta para sequence to sequence en el <a href='https://blog.keras.io/building-autoencoders-in-keras.html'>blog de keras </a> vamos a  proponer un encoder de dos layers LSTM y un decoder de una layer con una capa densa que permite recuperar las clases originales.

En este apartado, vamos a trabajar con una representación distinta de los datos: podemos afirmar que las diferentes notas son categorías completamente divergente, es decir, no mantienen una relación de ordinalidad entre ellas. Una representación que conserva esta diferencia es agregar al resultado del LabelEncoder una nueva dimensión que codifique con OneHotEncoding las distintas notas posibles. 



In [ ]:
def auto_encoder_model(sequence_len, input_dim, latent_dim):
    # Encoder
    inputs = Input(shape=(sequence_len, input_dim))
    encoded = LSTM(input_dim*2, return_sequences=True)(inputs)
    encoded = LSTM(latent_dim)(encoded)
    
    # Decoder
    decoded = RepeatVector(sequence_len)(encoded)
    decoded = LSTM(input_dim*2, return_sequences=True)(decoded)
    decoded = Dense(nclasses, activation='softmax')(decoded)
    auto_encoder_model = Model(inputs, decoded)
    auto_encoder_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    auto_encoder_model.summary()
    return auto_encoder_model
    #encoder = Model(inputs, encoded)

In [ ]:
input_ids = len(le.classes_)*array_input

In [ ]:
## VERSION ANTERIOR - BORRAR

def get_one_hot(input_array, nclasses):
    # Mapea los valores de input_array en tantas columnas como indica nclasses. 
    # Aumenta la dimensión del array de entrada agregando una dimensión para las columnas dummy.
    one_hot = tf.one_hot(input_array, nclasses)
    with tf.Session() as sess:
        return sess.run(one_hot)

one_hot_input_sequences = np.squeeze(get_one_hot(input_ids,len(le.classes_)))

In [ ]:
# OneHotEncoding
one_hot_input_sequences = tf.squeeze(tf.one_hot(input_ids, len(le.classes_))).numpy()

In [ ]:
one_hot_input_sequences.shape

(18335, 10, 256)

In [ ]:
sequence_len = timesteps

In [ ]:
latent_dim = 5

In [ ]:
nclasses = len(le.classes_)

In [ ]:
model = auto_encoder_model(sequence_len, nclasses, latent_dim)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 512)           1574912   
_________________________________________________________________
lstm_2 (LSTM)                (None, 5)                 10360     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 10, 5)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 512)           1060864   
_________________________________________________________________
dense_1 (Dense)              (None, 10, 256)           131328    
Total params: 2,777,464
Trainable params: 2,777,464
Non-trainable params: 0
_________________________________________________

In [ ]:
model.fit(one_hot_input_sequences, one_hot_input_sequences, epochs=150, batch_size=128, shuffle=True)

Epoch 1/150
18335/18335 [==============================] - 20s 1ms/step - loss: 3.8277e-04
Epoch 2/150
18335/18335 [==============================] - 19s 1ms/step - loss: 3.7484e-05
Epoch 3/150
18335/18335 [==============================] - 20s 1ms/step - loss: 1.3184e-06
Epoch 4/150
18335/18335 [==============================] - 20s 1ms/step - loss: 9.5475e-08
Epoch 5/150
18335/18335 [==============================] - 19s 1ms/step - loss: 3.3425e-08
Epoch 6/150
18335/18335 [==============================] - 20s 1ms/step - loss: 1.8752e-08
Epoch 7/150
18335/18335 [==============================] - 20s 1ms/step - loss: 1.2701e-08
Epoch 8/150
18335/18335 [==============================] - 20s 1ms/step - loss: 9.4882e-09
Epoch 9/150
18335/18335 [==============================] - 20s 1ms/step - loss: 7.5579e-09
Epoch 10/150
18335/18335 [==============================] - 20s 1ms/step - loss: 6.2482e-09
Epoch 11/150
18335/18335 [==============================] - 20s 1ms/step - loss: 5.3187e-

In [ ]:
def saveModel(model):
    model_json = model.to_json()
    with open("LSTMAutoencoder.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("LSTMAutoencoder.h5")

In [ ]:
saveModel(model)

Usamos el modelo para predecir uno de los propios datos de entrada. Observamos que la secuencia se reconstruye casi de forma perfecta. 

In [ ]:
from keras.models import model_from_json

def load_model(fn_json, fn_h5):
    with open(fn_json) as f:
        model = model_from_json(f.read())
    model.load_weights(fn_h5)
    return model

model = load_model("LSTMAutoencoder.json", "LSTMAutoencoder.h5")

In [ ]:
reconstruct =  model.predict(np.expand_dims(one_hot_input_sequences[0], axis=0))

In [ ]:
# La nueva secuencia de notas
np.argmax(reconstruct[0],axis=1)

array([253, 149, 219,  76, 192, 216, 216, 199, 199, 199])

In [ ]:
# La secuencia anterior de notas
np.argmax(one_hot_input_sequences[0],axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
def get_midi(note_chord_array, file_name):
    offset = 0
    output_notes = []
    # create note and chord objects based on the values generated by the model
    for pattern in note_chord_array:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.7
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(file_name))

In [ ]:
get_midi(le.classes_[np.argmax(reconstruct[0],axis=1)],'SecuenciaReconstruidaLSTM')

In [ ]:
get_midi(le.classes_[np.argmax(one_hot_input_sequences[0],axis=1)],'SecuenciaOriginal')

### Usando el modelo para generar datos nuevos

El modelo fue entrenado usando secuencias que aparecen en la música de Bach. Lo que hacemos a continuación es partir de una secuencia aleatoria y utilizar la red para generar una secuencia de música nueva. 

In [ ]:
#random_seq = get_one_hot(np.random.randint((nclasses - 1), size=10), nclasses)
random_seq = (tf.one_hot(np.random.randint((nclasses - 1), size=10), nclasses)).numpy()
muestra = le.classes_[np.argmax(random_seq, axis=1)]
print(muestra)
print('############################')
get_midi(muestra, 'Random')
long_sequence = [random_seq]
range_ = 10
for i in range(range_):
    new_seq = model.predict(long_sequence[-1].reshape(1, sequence_len, nclasses))[0]
    long_sequence.append(new_seq)
    
long_sequence = np.array(list(reduce(lambda x,y: np.concatenate((x,y)), long_sequence)))
output = le.classes_[np.argmax(long_sequence, axis=1)][range_:]
print(output)
get_midi(output, 'GeneradaLSTM')

['2.6.7' 'B5' '2.8' '5.8' '11.2.5.7' '8.11.2' '1.3.6' 'G3' '1.4' '2.4']
############################
['6.9' 'F#4' 'B2' 'A4' 'A4' '4.7' '11.3' '9.0.4' '10.2.5' '10.2.5' '6.9'
 '9' 'A4' '7.10' '5.11' '5.11' '5.11' 'A4' '1.4' '2.4' 'G4' 'A4' 'G#3'
 '7.0' '7.0' 'C5' 'C5' '7.11' 'C5' 'F4' 'G4' '2.5' 'B-4' 'B-4' '5' 'C5'
 'A4' 'D5' '7.10' 'B-4' 'G4' 'A4' 'C5' '0.3' 'C5' 'B-4' 'A4' 'F4' 'B-4'
 'D5' 'C5' 'B-4' 'D5' 'C5' 'B-4' 'A4' 'A4' 'B-4' 'F5' 'D5' 'C5' 'B-4' 'D5'
 'C5' 'D5' 'A4' 'B-4' 'G4' 'G4' 'F4' 'C5' 'B-4' 'C5' 'C5' 'B-4' 'A4' 'A4'
 'G4' 'G4' 'F5' 'C5' 'B-4' 'C5' 'B-4' 'B-4' 'A4' 'B-4' 'B-4' 'A4' 'A4'
 'D5' 'D5' 'B-4' 'B-4' 'B-4' 'A4' 'B-4' 'C5' 'A4' 'A4']
